In [1]:
import polars as pl

%load_ext autoreload
%autoreload 2

In [3]:
df = pl.read_csv("/home/grantham/Desktop/windmark/data/transactions.v1.csv")

df.columns = [col.lower().replace("?", "").replace(" ", "_") for col in df.columns]

In [4]:
(
    df.select(pl.all(), timeparts=pl.col("time").str.split(":"))
    .select(
        pl.all(),
        hour=pl.col("timeparts").list.first(),
        minute=pl.col("timeparts").list.last(),
    )
    .select(
        pl.all(),
        timestamp=pl.datetime(pl.col("year"), pl.col("month"), pl.col("day"), pl.col("hour"), pl.col("minute")),
    )
    .select(
        "use_chip",
        "merchant_state",
        "mcc",
        "timestamp",
        amount=pl.col("amount").str.strip_prefix("$").str.to_decimal(),
        merchant_name=pl.col("merchant_name").cast(pl.Utf8),
        target=pl.col("is_fraud"),
        event_id=pl.col("timestamp").cast(pl.Utf8),
        sequence_id=pl.col("user"),
    )
    .sort("timestamp")
    .select(
        pl.all(),
        event_order=pl.col("event_id").cum_count().over("sequence_id"),
        timedelta=pl.col("timestamp").sub(pl.col("timestamp").shift()).dt.seconds(),
    )
    .write_parquet("ledger.parquet")
)

/tmp/ipykernel_1623997/2848091057.py:27: DeprecationWarning: `seconds` is deprecated. It has been renamed to `total_seconds`.
  timedelta=pl.col("timestamp").sub(pl.col("timestamp").shift()).dt.seconds(),


In [7]:
dicts = [{"a": 1}, {"b": 2}, {"c": 3}, {}, {}]

In [11]:
dicts

[{}, {}, {}, {}, {}]

TypeError: dict expected at most 1 argument, got 5

{'a': 1, 'b': 2, 'c': 3}